First, import needed modules

In [20]:
import numpy as np
import pandas as pd
import geopandas as gpd
from geopandas.tools import geocode
from shapely.geometry import Point, LineString, Polygon
import shapely.speedups
from geopy.geocoders import Nominatim
import os

Initalize variables

In [21]:
path = os.getcwd()
path

'/Users/jan/Dropbox/UP_EPQM/2222/MA/powerlinemonsters'

Load Dataset which has the endpoints of power line projects

In [22]:
powerlines = pd.read_csv(f'{path}/data/geodata/powerlines.csv', sep=';')
#powerlines = powerlines.set_index("# BBPlG 2013", drop = False)
powerlines.head()

,# BBPlG 2013,Start,End,New,Retro
0,1,Emden-Borssum,"52.20189, 7.03448",1.0,NaN
1,1,"52.20189, 7.03448",Osterath,1.0,NaN
2,2,Osterath,Philippsburg,1.0,NaN
3,3,Brunsbüttel,Aub,1.0,NaN
4,3,Aub,Großgartach,1.0,NaN


Get the geocoordinates of the 'endpoint' cities using OSM/Nominatim API

In [23]:
start_locations = geocode(powerlines['Start'], provider='nominatim', user_agent='plm', timeout=4)
start_locations

,geometry,address
0,POINT (7.22429 53.34982),"Emden-Borssum, Birkenweg, Borssum, Emden, Nied..."
1,POINT (7.03585 52.20109),"Harberskamp, Morgensternsiedlung, Riekenhof, G..."
2,POINT (6.62072 51.26909),"Osterath, Meerbusch, Rhein-Kreis Neuss, Nordrh..."
3,POINT (9.13954 53.89725),"Brunsbüttel, Dithmarschen, Schleswig-Holstein,..."
4,POINT (10.06529 49.55250),"Aub, Verwaltungsgemeinschaft Aub, Landkreis Wü..."
...,...,...
59,POINT (9.07063 54.08988),"Meldorf, Mitteldithmarschen, Dithmarschen, Sch..."
60,POINT (7.20583 53.36705),"Emden, Niedersachsen, 26721, Deutschland"
61,POINT (8.64012 48.89101),"Höhenstraße, Birkenfeld, Enzkreis, Baden-Württ..."
62,POINT (10.20124 47.92271),"Woringen, Bad Grönenbach, Landkreis Unterallgä..."


In [24]:
end_locations = geocode(powerlines['End'], provider='nominatim', user_agent='plm', timeout=4)
end_locations

,geometry,address
0,POINT (7.03585 52.20109),"Harberskamp, Morgensternsiedlung, Riekenhof, G..."
1,POINT (6.62072 51.26909),"Osterath, Meerbusch, Rhein-Kreis Neuss, Nordrh..."
2,POINT (8.45467 49.23675),"Philippsburg, Verwaltungsverband Philippsburg,..."
3,POINT (10.06529 49.55250),"Aub, Verwaltungsgemeinschaft Aub, Landkreis Wü..."
4,POINT (9.12587 49.14254),"Großgartach, Leingarten, Landkreis Heilbronn, ..."
...,...,...
59,POINT (8.85870 54.12985),"Büsum, Büsum-Wesselburen, Dithmarschen, Schles..."
60,POINT (8.05776 53.32577),"Conneforde, Wiefelstede, Landkreis Ammerland, ..."
61,POINT (8.84929 48.94873),"Mühlacker, VVG der Stadt Mühlacker, Enzkreis, ..."
62,POINT (10.18132 47.98677),"Memmingen, Bayern, 87700, Deutschland"


Create powerlines_geo dataset by merging start and endpoints with the powerlines df

In [25]:
powerlines_geo_start = start_locations.join(powerlines, how = 'left')
powerlines_geo_start.rename(columns = {'geometry':'Start_geometry', 'address':'Start_address'}, inplace = True)
powerlines_geo = end_locations.join(powerlines_geo_start, how = 'left')
powerlines_geo.rename(columns = {'geometry':'End_geometry', 'address':'End_address'}, inplace = True)
powerlines_geo = powerlines_geo.reindex(columns=['# BBPlG 2013', 'Start', 'Start_address', 'Start_geometry', 'End', 'End_address', 'End_geometry', 'direct_line', 'New', 'Retro'])
powerlines_geo.to_csv(f'{path}/data/powerlinesgeo.csv')
powerlines_geo

,# BBPlG 2013,Start,Start_address,Start_geometry,End,End_address,End_geometry,direct_line,New,Retro
0,1,Emden-Borssum,"Emden-Borssum, Birkenweg, Borssum, Emden, Nied...",POINT (7.22429 53.34982),"52.20189, 7.03448","Harberskamp, Morgensternsiedlung, Riekenhof, G...",POINT (7.03585 52.20109),NaN,1.0,NaN
1,1,"52.20189, 7.03448","Harberskamp, Morgensternsiedlung, Riekenhof, G...",POINT (7.03585 52.20109),Osterath,"Osterath, Meerbusch, Rhein-Kreis Neuss, Nordrh...",POINT (6.62072 51.26909),NaN,1.0,NaN
2,2,Osterath,"Osterath, Meerbusch, Rhein-Kreis Neuss, Nordrh...",POINT (6.62072 51.26909),Philippsburg,"Philippsburg, Verwaltungsverband Philippsburg,...",POINT (8.45467 49.23675),NaN,1.0,NaN
3,3,Brunsbüttel,"Brunsbüttel, Dithmarschen, Schleswig-Holstein,...",POINT (9.13954 53.89725),Aub,"Aub, Verwaltungsgemeinschaft Aub, Landkreis Wü...",POINT (10.06529 49.55250),NaN,1.0,NaN
4,3,Aub,"Aub, Verwaltungsgemeinschaft Aub, Landkreis Wü...",POINT (10.06529 49.55250),Großgartach,"Großgartach, Leingarten, Landkreis Heilbronn, ...",POINT (9.12587 49.14254),NaN,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...
59,33,Meldorf,"Meldorf, Mitteldithmarschen, Dithmarschen, Sch...",POINT (9.07063 54.08988),Büsum,"Büsum, Büsum-Wesselburen, Dithmarschen, Schles...",POINT (8.85870 54.12985),NaN,1.0,NaN
60,34,Emden,"Emden, Niedersachsen, 26721, Deutschland",POINT (7.20583 53.36705),Conneforde,"Conneforde, Wiefelstede, Landkreis Ammerland, ...",POINT (8.05776 53.32577),NaN,NaN,1.0
61,35,"48.89024, 8.64165","Höhenstraße, Birkenfeld, Enzkreis, Baden-Württ...",POINT (8.64012 48.89101),Mühlacker,"Mühlacker, VVG der Stadt Mühlacker, Enzkreis, ...",POINT (8.84929 48.94873),NaN,1.0,NaN
62,36,Woringen,"Woringen, Bad Grönenbach, Landkreis Unterallgä...",POINT (10.20124 47.92271),Memmingen,"Memmingen, Bayern, 87700, Deutschland",POINT (10.18132 47.98677),NaN,NaN,1.0


Create shapefiles from the start and endpoints dfs

In [26]:
powerlines_geo_end = end_locations.join(powerlines, how = 'left')
powerlines_geo_end = powerlines_geo_end.to_crs(epsg=25832) #convert CRS to match BKG data
powerlines_geo_end.to_file(f'{path}/data/geodata/endpoints.shp')
powerlines_geo_start.rename(columns = {'Start_geometry':'geometry', 'Start_address':'address'}, inplace = True)
powerlines_geo_start = powerlines_geo_start.to_crs(epsg=25832)
powerlines_geo_start.to_file(f'{path}/data/geodata/startpoints.shp')

/opt/miniconda3/envs/vscode/lib/python3.10/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
/var/folders/wm/mtrphj0s0msgrmyshf3hgk740000gn/T/ipykernel_96084/1979668051.py:3: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  powerlines_geo_end.to_file(f'{path}/data/geodata/endpoints.shp')
/opt/miniconda3/envs/vscode/lib/python3.10/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
/var/folders/wm/mtrphj0s0msgrmyshf3hgk740000gn/T/ipykernel_96084/1979668051.py:6: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  powerlines_geo_start.to_file(f'{path}/data/geodata/startpoint

Create direct line from start and end coordinates

In [27]:
for i in range (0, len(powerlines_geo)):
    start = powerlines_geo['Start_geometry'].loc[powerlines_geo.index[i]]
    end = powerlines_geo['End_geometry'].loc[powerlines_geo.index[i]]
    powerlines_geo.loc[powerlines_geo.index[i], 'direct_line'] = LineString([start, end])
powerlines_geo.head()

/opt/miniconda3/envs/vscode/lib/python3.10/site-packages/pandas/core/internals/blocks.py:930: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  return self.coerce_to_target_dtype(value).setitem(indexer, value)
/opt/miniconda3/envs/vscode/lib/python3.10/site-packages/pandas/core/internals/managers.py:304: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  applied = getattr(b, f)(**kwargs)


,# BBPlG 2013,Start,Start_address,Start_geometry,End,End_address,End_geometry,direct_line,New,Retro
0,1,Emden-Borssum,"Emden-Borssum, Birkenweg, Borssum, Emden, Nied...",POINT (7.22429 53.34982),"52.20189, 7.03448","Harberskamp, Morgensternsiedlung, Riekenhof, G...",POINT (7.03585 52.20109),"LINESTRING (7.22428778316919 53.349817, 7.0358...",1.0,NaN
1,1,"52.20189, 7.03448","Harberskamp, Morgensternsiedlung, Riekenhof, G...",POINT (7.03585 52.20109),Osterath,"Osterath, Meerbusch, Rhein-Kreis Neuss, Nordrh...",POINT (6.62072 51.26909),"LINESTRING (7.03585 52.2010851, 6.6207151 51.2...",1.0,NaN
2,2,Osterath,"Osterath, Meerbusch, Rhein-Kreis Neuss, Nordrh...",POINT (6.62072 51.26909),Philippsburg,"Philippsburg, Verwaltungsverband Philippsburg,...",POINT (8.45467 49.23675),"LINESTRING (6.6207151 51.2690922, 8.4546679 49...",1.0,NaN
3,3,Brunsbüttel,"Brunsbüttel, Dithmarschen, Schleswig-Holstein,...",POINT (9.13954 53.89725),Aub,"Aub, Verwaltungsgemeinschaft Aub, Landkreis Wü...",POINT (10.06529 49.55250),"LINESTRING (9.1395423 53.8972549, 10.0652861 4...",1.0,NaN
4,3,Aub,"Aub, Verwaltungsgemeinschaft Aub, Landkreis Wü...",POINT (10.06529 49.55250),Großgartach,"Großgartach, Leingarten, Landkreis Heilbronn, ...",POINT (9.12587 49.14254),"LINESTRING (10.0652861 49.5524989, 9.1258701 4...",1.0,NaN


Export shapefile

In [28]:
powerlines_geo.rename(columns = {'direct_line':'geometry'}, inplace = True)
powerlines_geo = powerlines_geo.reindex(columns=['#BBPlG2013', 'Start', 'End', 'geometry', 'New', 'Retro'])
powerlines_geo = powerlines_geo.set_crs(epsg=4326)
powerlines_geo = powerlines_geo.to_crs(epsg=25832)
powerlines_geo.to_file(f'{path}/data/geodata/powerlines.shp')

/opt/miniconda3/envs/vscode/lib/python3.10/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
